In [ ]:
import numpy as np
import math
import os
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [ ]:
# root mean squared error
def RMSE(y, y_predict):
    return math.sqrt(np.sum((y - y_predict)**2)/len(y))  


# load training data
train_data = np.loadtxt('train/train_data.dat')

In [ ]:
order = 3
points = 101
alpha = np.linspace(0.001, 0.011, points)

RMSE_test = np.zeros(points)

RMSE_test_min = 100.
alpha_min  = 0

k = 5
kfold = KFold(n_splits = k, shuffle = True, random_state = 440)

# radius mean
for a in range(0, points):
    
    X = train_data[:,0:15].reshape(-1,15)
    y = train_data[:,-2].reshape(-1,1)
        
    RMSE_test_sum = 0
    
    for train_index, test_index in kfold.split(X):
        X_train = X[train_index]
        y_train = y[train_index]
        
        X_test  = X[test_index]
        y_test  = y[test_index]
        
        regression_mean = Pipeline([('scale', StandardScaler()), 
                                ('poly', PolynomialFeatures(order)),
                                ('reg', Lasso(alpha = alpha[a]))])
        
        regression_mean.fit(X_train, y_train)
        
        y_test_predict = regression_mean.predict(X_test).reshape(-1,1)

        RMSE_test_sum += RMSE(y_test, y_test_predict)
    
    RMSE_test[a] = RMSE_test_sum / k
    
    if RMSE_test[a] < RMSE_test_min:
        RMSE_test_min = RMSE_test[a]
        alpha_min = alpha[a]


mean_radius_RMSE = RMSE_test_min
mean_radius_alpha = alpha_min

plt.figure(figsize = (5,5))
plt.plot(alpha, RMSE_test, color = 'red')
plt.show()

RMSE_test_min = 100.
alpha_min  = 0

# radius std
for a in range(0, points):
    
    X = train_data[:,0:15].reshape(-1,15)
    y = train_data[:,-1].reshape(-1,1)
        
    RMSE_test_sum = 0
    
    for train_index, test_index in kfold.split(X):
        X_train = X[train_index]
        y_train = y[train_index]
        
        X_test  = X[test_index]
        y_test  = y[test_index]
        
        regression_std = Pipeline([('scale', StandardScaler()), 
                                ('poly', PolynomialFeatures(order)),
                                ('reg', Lasso(alpha = alpha[a]))])
        
        regression_std.fit(X_train, y_train)
        
        y_test_predict = regression_std.predict(X_test).reshape(-1,1)

        RMSE_test_sum += RMSE(y_test, y_test_predict)
    
    RMSE_test[a] = RMSE_test_sum / k
    
    if RMSE_test[a] < RMSE_test_min:
        RMSE_test_min = RMSE_test[a]
        alpha_min = alpha[a]
            

std_radius_RMSE = RMSE_test_min
std_radius_alpha = alpha_min

plt.figure(figsize = (5,5))
plt.plot(alpha, RMSE_test,  color = 'blue')
plt.show()

print("mean_test_RMSE = %.6f for alpha = %.4f" % (mean_radius_RMSE, mean_radius_alpha))
print("std_test_RMSE  = %.6f for alpha = %.4f" % (std_radius_RMSE, std_radius_alpha))

In [ ]:
# train regression model for mean fireball radius
#
# hyperparameters
alpha = mean_radius_alpha

# train regression model on all the data
X_train = train_data[:,0:15].reshape(-1,15)
y_train = train_data[:,-2].reshape(-1,1)

regression_mean = Pipeline([('scale', StandardScaler()), 
                            ('poly', PolynomialFeatures(order)),
                            ('reg', Lasso(alpha = alpha))])

regression_mean.fit(X_train, y_train)

In [ ]:
# train regression model for std fireball radius
#
# hyperparameters
alpha = std_radius_alpha

# train regression model on all the data
X_train = train_data[:,0:15].reshape(-1,15)
y_train = train_data[:,-1].reshape(-1,1)

regression_std = Pipeline([('scale', StandardScaler()),
                           ('poly', PolynomialFeatures(order)),
                           ('reg', Lasso(alpha = alpha))])

regression_std.fit(X_train, y_train)

In [ ]:
# test if auto grid size is large enough to fit all fireball radius in launch samples
#
# auto grid size without buffer (RMSE already included in mean, std)
#
def auto_grid_size(mean, std, number_sigmas, margin):
    return 2 * (mean  +  number_sigmas * std  + margin)


number_files = len(os.listdir('launch_fixed_grid/random_model_parameters'))

number_sigmas = 2    # number of sigmas
margin        = 1    # additional margin [fm] on each side of fireball

total   = 0
success = 0
average_area = 0
average_margin = 0

for n in range (0, number_files):
    parameters = np.loadtxt('launch_fixed_grid/random_model_parameters/'
                            + 'model_parameters_' + str(n + 1) + '.dat').reshape(1,-1)
    
    mean = regression_mean.predict(parameters)[0] + mean_radius_RMSE
    std  = regression_std.predict(parameters)[0]  + std_radius_RMSE
        
    L = auto_grid_size(mean, std, number_sigmas, margin)
    average_area += L * L
    
    fireball_radius = np.loadtxt('launch_fixed_grid/fireball_radius/'
                                 + '/fireball_radius_' + str(n + 1) + '.dat')[:,-1]
    
    for radius in fireball_radius:
        total += 1

        if L > 2*radius:
            success += 1
            average_margin += (L - 2*radius) / 2


average_area /= number_files
average_margin /= success
success_rate = 100. * success / total

print("Auto grid with %.1f sigmas and %.1f fm extra margin was able to fit %ld / %ld fireball samples" 
      % (number_sigmas, margin, success, total))
print()
print("Success rate   = %.2f" % success_rate, "%")
print("Average margin = %.2f fm" % average_margin)
print()
print("Average auto grid area = %.0f fm^2" % average_area)
print("Large fixed grid area  = 900 fm^2")


In [ ]:
np.savetxt('train/mean_radius_alpha_RMSE.dat', [[mean_radius_RMSE, mean_radius_alpha]])
np.savetxt('train/std_radius_alpha_RMSE.dat',  [[std_radius_RMSE,  std_radius_alpha]])